# Spectral leakage - non-periodic signals
In many real world applications, we do not have measurements that are perfectly periodic.
Consider the the following signal $\sin(2\pi t)$, $t\in[0, T]$.
If $T=[1,2,\dots]$, the signal is perfectly periodic.
However, changing the period of the signal such that the sampled data is not periodic, results in smearing of the FFT amplitudes.
To counteract this behavior, we can use [windowing](https://en.wikipedia.org/wiki/Window_function).
We use the Hamming window to illustrate this for the signal $x(t)=\cos(2\pi f t), t\in[0, T)$

In [1]:
from ipywidgets import widgets
import numpy as np
import matplotlib.pyplot as plt

@widgets.interact(
    T=widgets.FloatSlider(5.3, min=5, max=10), sampling_rate=(10, 30), frequency=(1,5))
def FFT_hamming(T:float, sampling_rate:int, frequency:float):
    ts = np.linspace(0, T, int(T*sampling_rate), endpoint=False)
    signal = np.cos(2*np.pi*ts*frequency)

    hamming = np.hamming(len(signal))
    plt.figure()
    plt.subplot(211)
    plt.plot(ts, hamming, "-ms", label="Hamming window")
    plt.plot(ts, hamming*signal, "--bo", label="Hamming Signal")
    plt.plot(ts, signal, "--r", label="Signal")
    plt.legend()

    fft_ham = np.fft.rfft(hamming*signal, n=len(signal))
    frequencies_ham = np.fft.rfftfreq(len(signal), 1./sampling_rate)
    plt.subplot(212)

    fft = np.fft.rfft(signal, n=len(signal))
    frequencies = np.fft.rfftfreq(len(signal), 1./sampling_rate)

    plt.stem(frequencies, np.abs(fft)/len(signal), linefmt="r-", markerfmt="rs", basefmt="r", label="FFT")
    plt.stem(frequencies_ham, np.abs(fft_ham)/len(signal), linefmt="b-", markerfmt="bo", basefmt="b",
     label="FFT Hamming")

    plt.legend()
    plt.grid()
    plt.show()



interactive(children=(FloatSlider(value=5.3, description='T', max=10.0, min=5.0), IntSlider(value=20, descript…

We can look at this combined with noise in the following widget for the signal
$$
x(t) = 2.3 \cos(2\pi t) + 4 \sin(12\pi t) + A \cdot noise
$$

In [13]:
ws = {'description_width': 'initial'}

def generate_signal_and_fft(T:float, use_hamming:bool, sampling_rate:int, noise:bool):
    t = np.linspace(0, T, int(T*sampling_rate), endpoint=False)
    N = len(t)
    signal = 2.3 * np.cos(2*np.pi*t) + 4 * np.sin(2*np.pi*6*t)
    hamming = np.hamming(N) if use_hamming else np.ones(N)
    noise = np.float64(noise)*np.random.randn(N)
    windowed_signal = (signal + noise)*hamming
    fft = np.fft.rfft(windowed_signal, n=N)
    frequencies = np.fft.rfftfreq(len(windowed_signal), d=1./sampling_rate)
    return (t, signal + noise, windowed_signal), (frequencies, fft)

@widgets.interact(
    use_hamming=widgets.Checkbox(value=True, description="Hamming"),
    noisy_signal=widgets.Checkbox(value=False, description="Noise"),
    sampling_rate=widgets.ToggleButtons(options=[16, 32, 64],description="Sampling rate (Hz)",
    style=ws),
    T=widgets.FloatSlider(1.3, min=1, max=2, description="Simulation time (s)",style=ws))
def FFT(sampling_rate:int, T:float, use_hamming:bool, noisy_signal:bool):
    (t, signal, wsignal), (frequencies, fft) = generate_signal_and_fft(T, noise=noisy_signal,
                                                              use_hamming=use_hamming,
                                                              sampling_rate=sampling_rate)
    plt.figure(figsize=(15,8))
    plt.subplot(121)
    plt.title("Signal")
    plt.gca().set_xlabel("t")
    plt.gca().set_ylabel("x(t)")
    plt.plot(t,signal, label="Signal")
    if use_hamming:
        plt.plot(t, wsignal, label="Windowed singal")
    plt.grid()
    plt.legend()
    plt.subplot(122)
    plt.title("Power Spectrum Density")
    plt.stem(frequencies, np.abs(fft)**2)
    plt.gca().set_xlabel("f")
    plt.gca().set_ylabel("PSD")
    plt.gca().set_yscale("log")
    plt.title("")
    plt.grid()
    plt.show()


interactive(children=(ToggleButtons(description='Sampling rate (Hz)', options=(16, 32, 64), style=ToggleButton…